# IMPORT

In [ ]:

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk

from tkinter import ttk
from tkinter import *

# Load Datasets

In [ ]:
data_path = 'datasets/Datafull train.csv'

df = pd.read_csv(data_path, delimiter=";")

df.head()

In [ ]:
# cek dimensi data sebelum masuk data preprocessing
df.shape

In [ ]:
df.nunique() # mengecek data unique pada setiap kolom

In [ ]:
df.info()

In [ ]:
df.dtypes

# Data PreProcessing

In [ ]:
df.isna().sum() # mengecek data yang kosong
df.duplicated().sum # mengecek data yang duplikat

df = df.drop_duplicates() #menghapus data duplikat
df = df.fillna(df.mode().iloc[0])

In [ ]:
df.shape # cek dimensi data setelah dihapus data duplikat

# Exploratory Data Analys

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(data=df, x="char")
plt.title("Distribusi Label / Char")
plt.ylabel("Jumlah Koordinat")
plt.show()

In [ ]:
def plot_sample(row): 

    x = [row[f"{i}x"] for i in range(21)]
    y = [row[f"{i}y"] for i in range(21)]

    plt.Figure(figsize=(10, 8))
    plt.plot(x, y, marker="o")
    plt.gca().invert_yaxis()
    plt.title(f"Class: {row['char']}")
    plt.show()

In [ ]:
for label in df['char'].unique():
    sample = df[df['char'] == label].iloc[0]
    plot_sample(sample)

In [ ]:
x_cols = [c for c in df.columns if c.endswith('x')]
y_cols = [c for c in df.columns if c.endswith('y')]

plt.figure(figsize=(12, 4))
sns.boxplot(data=df[x_cols])
plt.title("Distribusi Koordinat X")
plt.show()

plt.figure(figsize=(12, 4))
sns.boxplot(data=df[y_cols])
plt.title("Distribusi Koordinat Y")
plt.show()